In [ ]:
import numpy as np
import pandas as pd
import string
import math
import random
from sklearn import svm
from sklearn import ensemble
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/train.csv")
result = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/gender_submission.csv")
testing = pd.read_csv("/Users/Ellie/Documents/Kaggle/Titanic/test.csv")
df.head(5)

In [ ]:
# Getting substring for title
def getSubstring(string, substrings):
    for s in substrings:
        if string.find(s) != -1:
            return s
    return np.nan
       

# Replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title = x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
    
# Replacing missing value for age 
def replace_ages(x, ageGroup):
    age = x['Age']
    if math.isnan(age):
        return ageGroup[x['Pclass']][x['SibSp']]
    else:
        return age


In [ ]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer'] 
df['Title'] = df['Name'].map(lambda x: getSubstring(x, title_list))
df['Title'] = df.apply(replace_titles, axis = 1)
df['LastName'] = df['Name'].split(',')[1]

In [ ]:
# Fill the missing values
ageGroup = df.groupby(['Pclass', 'SibSp'])['Age'].mean()
ageGroup[3][8] = df['Age'][df['Pclass'] == 3].mean()
df['Age'] = df.apply(lambda x: replace_ages(x, ageGroup), axis = 1)

In [ ]:
df.isnull().any()
df['Name'].split(',')[1]

In [ ]:
# Replace carbin number
cabin_null = df['Cabin'].isnull()
cabin_list = [i for i in range(0, len(df['Cabin'])) if cabin_null[i] == True]
# df['Cabin'][cabin_list] = 'Unknown'
# cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
# df['Cabin']=df['Cabin'].map(lambda x: getSubstring(x, cabin_list))

In [ ]:
# Embark only 2 missing values
df = df[df['Embarked'].isnull() == False]
df = df.reset_index(drop=True)

In [ ]:
# Deal with the categorical data
df['Sex_male'] = (df['Sex'] == 'male').astype('int')
df['Title_Master'] = (df['Title'] == 'Master').astype('int')
df['Title_Miss'] = (df['Title'] == 'Miss').astype('int')
df['Title_Mr'] = (df['Title'] == 'Mr').astype('int')
df['Title_Mrs'] = (df['Title'] == 'Mrs').astype('int')
df['Cabin_A'] = (df['Cabin'] == 'A').astype('int')
df['Cabin_B'] = (df['Cabin'] == 'B').astype('int')
df['Cabin_C'] = (df['Cabin'] == 'C').astype('int')
df['Cabin_D'] = (df['Cabin'] == 'D').astype('int')
df['Cabin_E'] = (df['Cabin'] == 'E').astype('int')
df['Cabin_F'] = (df['Cabin'] == 'F').astype('int')
df['Cabin_G'] = (df['Cabin'] == 'G').astype('int')
df['Cabin_T'] = (df['Cabin'] == 'T').astype('int')
df['Cabin_Unknown'] = (df['Cabin'] == 'Unknown').astype('int')
df['Embarked_C'] = (df['Embarked'] == 'C').astype('int')
df['Embarked_Q'] = (df['Embarked'] == 'Q').astype('int')
df['Embarked_S'] = (df['Embarked'] == 'S').astype('int')


In [ ]:
# drop the unnecessary columns
data = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked', 'Sex', 'Survived', 'Title'], axis = 1)
response = df['Survived']

In [ ]:
# perform cross validation, 5 folds
cv_id = []
for i in range(0, 5):
    cv_id.extend([i] * 177)
cv_id.extend([0] * 4) 
random.shuffle(cv_id)

In [ ]:
# cross validation 
rate = []
for i in range(0,5):
    train_index = [index for index in range(0, len(df)) if cv_id[index] != i]
    test_index = [index for index in range(0, len(df)) if cv_id[index] == i]
    train = data.iloc[train_index]
    train_response = response.iloc[train_index]
    test = data.iloc[test_index]
    test_response = response.iloc[test_index]
#     clf = svm.LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=1e-5)
    clf = ensemble.RandomForestClassifier(n_estimators = 10)
    clf.fit(train, train_response)
    pred = clf.predict(test)
    rate.append(sum(pred == test_response) / len(test_response))
np.mean(rate)
    
    

    

In [ ]:
rate